In [1]:
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import osmnx as ox
import geopandas as gpd
import osmium
from shapely.geometry import Polygon
import json
import fiona
from shapely.geometry import shape 


for module in [pd, gpd]:
    print(module.__name__, module.__version__)

pandas 1.2.0
geopandas 0.9.0


In [2]:
pd.set_option('display.max_columns', None)

In [3]:
%%bash
wget https://download.bbbike.org/osm/extract/planet_-9.192,40.715_-7.611,41.555.osm.pbf \
    --quiet -O map_data/Porto.osm.pbf

In [4]:
%%bash
wget https://download.bbbike.org/osm/extract/planet_-9.89,38.265_-8.309,39.136.osm.pbf \
    --quiet -O map_data/Lisbon.osm.pbf

In [5]:
!ogrinfo map_data/Lisbon.osm.pbf

INFO: Open of `map_data/Lisbon.osm.pbf'
      using driver `OSM' successful.
1: points (Point)
2: lines (Line String)
3: multilinestrings (Multi Line String)
4: multipolygons (Multi Polygon)
5: other_relations (Geometry Collection)


In [6]:
%%bash
ogr2ogr -f "GPKG" \
    map_data/lisbon_points.gpkg \
    map_data/Lisbon.osm.pbf \
    POINTS \
    -nln all

0...10...20...30...40...50...60...70...80...90...100 - done.


In [7]:
gdf_lis_pts = gpd.read_file("map_data/lisbon_points.gpkg", drive = 'GPKG')
gdf_lis_pts

,osm_id,name,amenity,barrier,highway,ref,address,is_in,place,man_made,other_tags,geometry
0,20629108,Lapa da Serra,None,None,None,None,None,None,neighbourhood,None,None,POINT (-9.39964 38.95586)
1,20629118,None,None,None,crossing,None,None,None,None,None,None,POINT (-9.41390 38.96183)
2,21272086,None,None,None,crossing,None,None,None,None,None,"""crossing""=>""uncontrolled"",""crossing_ref""=>""ze...",POINT (-9.18832 38.74858)
3,21404046,None,None,None,bus_stop,None,None,None,None,None,None,POINT (-9.45296 38.75312)
4,21404049,None,None,None,bus_stop,None,None,None,None,None,None,POINT (-9.45655 38.75241)
...,...,...,...,...,...,...,...,...,...,...,...,...
99115,8641657617,None,None,None,None,None,None,None,None,windmill,None,POINT (-9.25846 38.83832)
99116,8641657618,None,None,None,None,None,None,None,None,windmill,None,POINT (-9.26068 38.83765)
99117,8641657619,None,None,None,None,None,None,None,None,windmill,None,POINT (-9.25440 38.84683)
99118,8641657620,None,None,None,None,None,None,None,None,windmill,None,POINT (-9.25396 38.84726)


In [8]:
gdf_lis_pts['highway'].unique()

array([None, 'crossing', 'bus_stop', 'motorway_junction',
       'traffic_signals', 'turning_circle', 'services', 'mini_roundabout',
       'speed_camera', 'traffic_signals;crossing', 'give_way', 'elevator',
       'stop', 'turning_loop', 'emergency_access_point', 'street_lamp',
       'trailhead', 'toll_gantry', 'rest_area', 'traffic_mirror',
       'platform', 'steps', 'raceway', 'footway', 'milestone', 'path',
       'crossing;traffic_signals'], dtype=object)

In [19]:
gdf_lis_pts['highway'].value_counts()

crossing                    14074
bus_stop                     4324
traffic_signals              2194
street_lamp                  1294
turning_circle               1253
give_way                      550
motorway_junction             337
stop                          290
mini_roundabout               128
speed_camera                   47
elevator                       44
turning_loop                   17
milestone                      14
traffic_signals;crossing       13
traffic_mirror                 11
services                        6
trailhead                       6
toll_gantry                     6
rest_area                       5
steps                           3
emergency_access_point          2
footway                         2
crossing;traffic_signals        2
raceway                         2
path                            1
platform                        1
Name: highway, dtype: int64

## Other Geometries

### Polygons - Ways

In [9]:
%%bash
ogr2ogr -f "GPKG" \
    map_data/lisbon_polygons.gpkg \
    map_data/Lisbon.osm.pbf \
    -nlt POLYGONS \
    -nln polygons

0...10...20...30...40...50...60...70...80...90...100 - done.


Warning 1: A geometry of type POINT is inserted into layer polygons of geometry type POLYGON, which is not normally allowed by the GeoPackage specification, but the driver will however do it. To create a conformant GeoPackage, if using ogr2ogr, the -nlt option can be used to override the layer geometry type. This warning will no longer be emitted for this combination of layer and feature geometry type.
Warning 1: A geometry of type LINESTRING is inserted into layer polygons of geometry type POLYGON, which is not normally allowed by the GeoPackage specification, but the driver will however do it. To create a conformant GeoPackage, if using ogr2ogr, the -nlt option can be used to override the layer geometry type. This warning will no longer be emitted for this combination of layer and feature geometry type.
Warning 1: A geometry of type MULTILINESTRING is inserted into layer polygons of geometry type POLYGON, which is not normally allowed by the GeoPackage specification, but the driver w

In [13]:
#Check Geometry
def isvalid(geom):
    try:
        shape(geom)
        return 1
    except:
        return 0

In [14]:
#Read data
layer_file = "map_data/lisbon_polygons.gpkg"
collection = list(fiona.open(layer_file,'r'))
df1 = pd.DataFrame(collection)

df1['isvalid'] = df1['geometry'].apply(lambda x: isvalid(x))
df1 = df1[df1['isvalid'] == 1]
collection = json.loads(df1.to_json(orient='records'))

#Convert to geodataframe
gdf_lis_poly = gpd.GeoDataFrame.from_features(collection)

In [17]:
gdf_lis_poly['highway'].nunique()

49